# Mini Read Me

* Possible need to clean the data
* Other advert related tweets are still inside (Forex Trading)
* How to remove hospital related tweets? Currently, any tweets mentioning hospital is deleted

In [1]:
# !pip install pyLDAvis # run this on first run. Then comment out
import pyLDAvis
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer # replace TfidfVectorizer with CountVectorizer if you want the count method
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation

In [2]:
input_file_path = "../../../data/twitter/cleaned/final_clean_data.csv"
df_raw = pd.read_csv(input_file_path).drop('Unnamed: 0',axis=1)
df_raw = df_raw.rename(columns={'search':'id_search'})

In [3]:
# Cleaning the searched keywords as it is causing problems with the pd.melt
df_raw['id_search'] = df_raw['id_search'].apply(lambda x: x.strip())
df_raw['id_search'] = df_raw['id_search'].apply(lambda x: x.replace(" ", "_"))        
# The first covid case is reported to be on Jan 23 2020
df_raw_pre_covid = df_raw[df_raw['date'] < '2020-01-23'].drop(['date', 
                                                               'hashtags',
                                                               'username',
                                                               'nlikes',
                                                               'nreplies',
                                                               'nretweets',
                                                               'year',
                                                               'month'],
                                                                axis=1)

df_raw_covid = df_raw[df_raw['date'] >= '2020-01-23'].drop(['date', 
                                                            'hashtags',
                                                            'username',
                                                            'nlikes',
                                                            'nreplies',
                                                            'nretweets',
                                                            'year',
                                                            'month'],
                                                            axis=1)


# Remove rows with these stopwords
search_words = ['nikerunning', 'dec', 'delavin',
                'kisses', 'agrunningmoms', 'migraineinsg',
                'jordan', 'munchausen', 'concert',
                'wspd', 'comedy', 'gentlemonstersunglasses', 
                'gentlemonster', 'youtube', 'star', 'place', 
                'comedy', 'poetry', 'want', 'know', 'diagrams', 
                'pared', 'owls', 'riser', 'street', 'style', 'buat',
                'phd', 'hi', 'im', 'gold', 'standard', 'yeah', 'watch', 
                'arena', 'things', 'teka', 'gt', 'matter', 'forex', 
                'zentrader', 'hospital', 'zerowaste', 'foodwaste', 
                'kangdaniel', 'danielk']

df_raw_pre_covid = (df_raw_pre_covid[~df_raw_pre_covid['clean_tweet4']
                                     .str.contains('|'.join(search_words), na=False)]) #ignore NaN else the code breaks

df_raw_covid = (df_raw_covid[~df_raw_covid['clean_tweet4']
                             .str.contains('|'.join(search_words), na=False)]) #ignore NaN else the code breaks


df_raw_pre_covid.dropna(axis=0,inplace=True)
df_raw_covid.dropna(axis=0,inplace=True)


# # Groups all the text in a keyword category to find common words associated with the keyword
# df_raw_pre_covid_group = df_raw_pre_covid.groupby(['id_search'], as_index=False)['clean_tweet4'].apply(' '.join)
# df_raw_covid_group = df_raw_covid.dropna().groupby(['id_search'], as_index=False)['clean_tweet4'].apply(' '.join)

# Unigram Pre-Covid

In [4]:
vectorizer = TfidfVectorizer(
ngram_range=(1, 1),
analyzer='word',       
max_features=5000)# max number of unique words
                            
data_matrix = vectorizer.fit_transform(df_raw_pre_covid['clean_tweet4'])

# Very slow!!!

lda_model = LatentDirichletAllocation(
n_components=10, # Number of topics
learning_method='online',
random_state=20,       
n_jobs = -1  # Use all available CPUs
                                     )
lda_output = lda_model.fit_transform(data_matrix)

import pyLDAvis.sklearn
pyLDAvis.enable_notebook()
pyLDAvis.sklearn.prepare(lda_model, data_matrix, vectorizer, mds='tsne')

/Users/jeromeyue/Desktop/omdena/lib/python3.9/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(
/Users/jeromeyue/Desktop/omdena/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/jeromeyue/Desktop/omdena/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/jeromeyue/Desktop/omdena/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/jeromeyue/Deskt

PreparedData(topic_coordinates=               x          y  topics  cluster       Freq
topic                                                  
6      32.420952   8.115230       1        1  23.298437
4       0.648955  71.707100       2        1  19.218606
0      71.019684   9.971006       3        1  13.381792
8       3.785981  33.185135       4        1   7.924214
1     -42.666500  -9.222070       5        1   7.090815
7      -3.621689 -43.526516       6        1   6.076542
5      -3.632402  -4.171708       7        1   5.869312
9     -36.584400  35.804413       8        1   5.781221
3      44.391380  50.615383       9        1   5.740761
2      40.235809 -31.591665      10        1   5.618300, topic_info=           Term        Freq       Total Category  logprob  loglift
4478      tired  347.000000  347.000000  Default  30.0000  30.0000
1369      fight  226.000000  226.000000  Default  29.0000  29.0000
909        dead  186.000000  186.000000  Default  28.0000  28.0000
4522      toxic  110.000000  110.000000  Default  27.0000  27.0000
2024   insomnia   44.000000   44.000000  Default  26.0000  26.0000
...         ...         ...         ...      ...      ...      ...
3182       post    5.725374   14.822702  Topic10  -5.3155   1.9279
4106     spread    6.194176   85.407500  Topic10  -5.2368   0.2553
3772        set    4.819657    8.150691  Topic10  -5.4877   2.3537
3902  singapore    5.562166   89.627270  Topic10  -5.3444   0.0995
3050     people    4.989873   67.417746  Topic10  -5.4530   0.2756

[503 rows x 6 columns], token_table=      Topic      Freq    Term
term                         
9         1  0.221599   abuse
9         6  0.177279   abuse
9         7  0.044320   abuse
9        10  0.576156   abuse
24        8  0.899081  across
...     ...       ...     ...
4996      1  0.342042    zoom
4996      2  0.105244    zoom
4996      4  0.105244    zoom
4996      6  0.105244    zoom
4996      8  0.342042    zoom

[684 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[7, 5, 1, 9, 2, 8, 6, 10, 4, 3])

# Bigram Pre-Covid

In [5]:
vectorizer = TfidfVectorizer(
ngram_range=(2, 2),
analyzer='word',       
max_features=5000)# max number of unique words
                            
data_matrix = vectorizer.fit_transform(df_raw_pre_covid['clean_tweet4'])

# Very slow!!!

lda_model = LatentDirichletAllocation(
n_components=10, # Number of topics
learning_method='online',
random_state=20,       
n_jobs = -1  # Use all available CPUs
                                     )
lda_output = lda_model.fit_transform(data_matrix)

import pyLDAvis.sklearn
pyLDAvis.enable_notebook()
pyLDAvis.sklearn.prepare(lda_model, data_matrix, vectorizer, mds='tsne')

/Users/jeromeyue/Desktop/omdena/lib/python3.9/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(
/Users/jeromeyue/Desktop/omdena/lib/python3.9/site-packages/sklearn/manifold/_t_sne.py:691: FutureWarning: 'square_distances' has been introduced in 0.24 to help phase out legacy squaring behavior. The 'legacy' setting will be removed in 1.1 (renaming of 0.26), and the default setting will be changed to True. In 1.3, 'square_distances' will be removed altogether, and distances will be squared by default. Set 'square_distances'=True to silence this warning.
  warnings.warn(


PreparedData(topic_coordinates=               x          y  topics  cluster       Freq
topic                                                  
6      32.420952   8.115230       1        1  10.498353
0       0.648955  71.707100       2        1  10.288246
9      71.019684   9.971006       3        1  10.103496
4       3.785981  33.185135       4        1  10.010662
8     -42.666500  -9.222070       5        1   9.976651
7      -3.621689 -43.526516       6        1   9.957810
5      -3.632402  -4.171708       7        1   9.954535
1     -36.584400  35.804413       8        1   9.825091
3      44.391380  50.615383       9        1   9.699917
2      40.235809 -31.591665      10        1   9.685239, topic_info=                Term       Freq      Total Category  logprob  loglift
2556       need help  27.000000  27.000000  Default  30.0000  30.0000
3887      sick tired  25.000000  25.000000  Default  29.0000  29.0000
2398   mental health  24.000000  24.000000  Default  28.0000  28.0000
2925     please help  27.000000  27.000000  Default  27.0000  27.0000
4066     spread love  18.000000  18.000000  Default  26.0000  26.0000
...              ...        ...        ...      ...      ...      ...
4011     someone pls   2.448177   3.024603  Topic10  -5.8615   2.1231
2260       love hate   2.437693   3.014119  Topic10  -5.8658   2.1223
2399  mental illness   4.371336   5.414957  Topic10  -5.2818   2.1205
1457       god tired   2.396172   2.972604  Topic10  -5.8830   2.1190
2115       like dead   4.616189   6.052834  Topic10  -5.2273   2.0636

[336 rows x 6 columns], token_table=      Topic      Freq            Term
term                                 
5         4  0.665365     abuse power
25        1  0.942863  actually tired
42        3  0.779220      ajak fight
69        4  0.962347    already dead
74        7  0.969388   already tired
...     ...       ...             ...
4832      6  0.986214       wear mask
4867     10  0.895549       went find
4876      6  0.918058       whole day
4971      8  0.849217        year old
4973      1  0.892931       years ago

[312 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[7, 1, 10, 5, 9, 8, 6, 2, 4, 3])

# Trigram Pre Covid

In [6]:
vectorizer = TfidfVectorizer(
ngram_range=(3, 3),
analyzer='word',       
max_features=5000)# max number of unique words
                            
data_matrix = vectorizer.fit_transform(df_raw_pre_covid['clean_tweet4'])

# Very slow!!!

lda_model = LatentDirichletAllocation(
n_components=10, # Number of topics
learning_method='online',
random_state=20,       
n_jobs = -1  # Use all available CPUs
                                     )
lda_output = lda_model.fit_transform(data_matrix)

import pyLDAvis.sklearn
pyLDAvis.enable_notebook()
pyLDAvis.sklearn.prepare(lda_model, data_matrix, vectorizer, mds='tsne')

/Users/jeromeyue/Desktop/omdena/lib/python3.9/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(
/Users/jeromeyue/Desktop/omdena/lib/python3.9/site-packages/sklearn/manifold/_t_sne.py:691: FutureWarning: 'square_distances' has been introduced in 0.24 to help phase out legacy squaring behavior. The 'legacy' setting will be removed in 1.1 (renaming of 0.26), and the default setting will be changed to True. In 1.3, 'square_distances' will be removed altogether, and distances will be squared by default. Set 'square_distances'=True to silence this warning.
  warnings.warn(


PreparedData(topic_coordinates=               x          y  topics  cluster       Freq
topic                                                  
0      32.420952   8.115230       1        1  10.306271
6       0.648955  71.707100       2        1  10.277457
4      71.019684   9.971006       3        1  10.274982
8       3.785981  33.185135       4        1  10.091583
5     -42.666500  -9.222070       5        1   9.991159
1      -3.621689 -43.526516       6        1   9.941283
3      -3.632402  -4.171708       7        1   9.851757
7     -36.584400  35.804413       8        1   9.817021
9      44.391380  50.615383       9        1   9.795048
2      40.235809 -31.591665      10        1   9.653439, topic_info=                     Term      Freq     Total Category  logprob  loglift
640       never get tired  7.000000  7.000000  Default  30.0000  30.0000
3845  quarter life crisis  4.000000  4.000000  Default  29.0000  29.0000
575      marina bay sands  3.000000  3.000000  Default  28.0000  28.0000
4865     tired cant sleep  3.000000  3.000000  Default  27.0000  27.0000
509    late night testing  4.000000  4.000000  Default  26.0000  26.0000
...                   ...       ...       ...      ...      ...      ...
77     bikini posing walk  0.867848  1.285078  Topic10  -6.1855   1.9453
178        dead body ever  0.854883  1.272113  Topic10  -6.2005   1.9404
545       look tired rest  0.849427  1.266657  Topic10  -6.2069   1.9383
166      daily sheet mask  0.845311  1.262549  Topic10  -6.2118   1.9367
32      apieu daily sheet  0.845311  1.262549  Topic10  -6.2118   1.9367

[334 rows x 6 columns], token_table=      Topic      Freq                          Term
term                                               
4         8  0.778371                 act like care
8         4  0.733706              address bio give
14        2  0.706865  allaboutfood foodie foodporn
15        8  0.840380              almost half dead
21        7  0.719139          always feeling tired
...     ...       ...                           ...
4957      6  0.522907                 wanna go home
4958      6  0.705449               wanna go lonely
4971      8  0.799864         without feeling tired
4975      1  0.706968         witnessing slow death
4976      4  0.692766               woke even tired

[304 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 7, 5, 9, 6, 2, 4, 8, 10, 3])

# Uni to Trigram Pre Covid
Let CountVectorizer sort itself out

In [7]:
vectorizer = TfidfVectorizer(
ngram_range=(1, 3),
analyzer='word',       
max_features=5000)# max number of unique words
                            
data_matrix = vectorizer.fit_transform(df_raw_pre_covid['clean_tweet4'])

# Very slow!!!

lda_model = LatentDirichletAllocation(
n_components=10, # Number of topics
learning_method='online',
random_state=20,       
n_jobs = -1  # Use all available CPUs
                                     )
lda_output = lda_model.fit_transform(data_matrix)

import pyLDAvis.sklearn
pyLDAvis.enable_notebook()
pyLDAvis.sklearn.prepare(lda_model, data_matrix, vectorizer, mds='tsne')

/Users/jeromeyue/Desktop/omdena/lib/python3.9/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(
/Users/jeromeyue/Desktop/omdena/lib/python3.9/site-packages/sklearn/manifold/_t_sne.py:691: FutureWarning: 'square_distances' has been introduced in 0.24 to help phase out legacy squaring behavior. The 'legacy' setting will be removed in 1.1 (renaming of 0.26), and the default setting will be changed to True. In 1.3, 'square_distances' will be removed altogether, and distances will be squared by default. Set 'square_distances'=True to silence this warning.
  warnings.warn(


PreparedData(topic_coordinates=               x          y  topics  cluster       Freq
topic                                                  
7      32.420952   8.115230       1        1  19.087020
9       0.648955  71.707100       2        1  13.755524
5      71.019684   9.971006       3        1  12.069494
8       3.785981  33.185135       4        1  10.331277
4     -42.666500  -9.222070       5        1   8.404105
6      -3.621689 -43.526516       6        1   8.212815
1      -3.632402  -4.171708       7        1   7.716979
0     -36.584400  35.804413       8        1   7.560219
3      44.391380  50.615383       9        1   7.043099
2      40.235809 -31.591665      10        1   5.819466, topic_info=              Term        Freq       Total Category  logprob  loglift
4420         tired  320.000000  320.000000  Default  30.0000  30.0000
1477         fight  215.000000  215.000000  Default  29.0000  29.0000
921           dead  174.000000  174.000000  Default  28.0000  28.0000
4541         toxic  106.000000  106.000000  Default  27.0000  27.0000
2665          mask   44.000000   44.000000  Default  26.0000  26.0000
...            ...         ...         ...      ...      ...      ...
1606       forward    4.720984    6.851306  Topic10  -5.5836   2.4715
847         crisis   10.622597   48.143350  Topic10  -4.7726   1.3328
2525        lonely    8.127052   34.533680  Topic10  -5.0404   1.3972
4729       variant    4.583195   10.311290  Topic10  -5.6132   2.0331
4577  transmission    4.375075   10.711210  Topic10  -5.6597   1.9486

[516 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
7         1  0.237573      abuse
7         2  0.760235      abuse
27        6  0.876591        act
40       10  0.841716        add
43        3  0.211633  addiction
...     ...       ...        ...
4996      6  0.025294       zoom
4996      7  0.404699       zoom
4996      8  0.025294       zoom
4996      9  0.075881       zoom
4996     10  0.050587       zoom

[791 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[8, 10, 6, 9, 5, 7, 2, 1, 4, 3])

# Unigram Covid

In [8]:
vectorizer = TfidfVectorizer(
ngram_range=(1, 1),
analyzer='word',       
max_features=5000)# max number of unique words
                            
data_matrix = vectorizer.fit_transform(df_raw_covid['clean_tweet4'])

# Very slow!!!

lda_model = LatentDirichletAllocation(
n_components=10, # Number of topics
learning_method='online',
random_state=20,       
n_jobs = -1  # Use all available CPUs
                                     )
lda_output = lda_model.fit_transform(data_matrix)

import pyLDAvis.sklearn
pyLDAvis.enable_notebook()
pyLDAvis.sklearn.prepare(lda_model, data_matrix, vectorizer, mds='tsne')

/Users/jeromeyue/Desktop/omdena/lib/python3.9/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(
/Users/jeromeyue/Desktop/omdena/lib/python3.9/site-packages/sklearn/manifold/_t_sne.py:691: FutureWarning: 'square_distances' has been introduced in 0.24 to help phase out legacy squaring behavior. The 'legacy' setting will be removed in 1.1 (renaming of 0.26), and the default setting will be changed to True. In 1.3, 'square_distances' will be removed altogether, and distances will be squared by default. Set 'square_distances'=True to silence this warning.
  warnings.warn(


PreparedData(topic_coordinates=               x          y  topics  cluster       Freq
topic                                                  
4      32.420952   8.115230       1        1  15.275589
6       0.648955  71.707100       2        1  14.652569
1      71.019684   9.971006       3        1  10.825094
7       3.785981  33.185135       4        1   9.868489
9     -42.666500  -9.222070       5        1   9.269745
5      -3.621689 -43.526516       6        1   9.202565
0      -3.632402  -4.171708       7        1   8.830276
8     -36.584400  35.804413       8        1   8.169714
3      44.391380  50.615383       9        1   7.126205
2      40.235809 -31.591665      10        1   6.779755, topic_info=          Term        Freq       Total Category  logprob  loglift
4998      zoom  275.000000  275.000000  Default  30.0000  30.0000
721    circuit  215.000000  215.000000  Default  29.0000  29.0000
512    breaker  214.000000  214.000000  Default  28.0000  28.0000
3181  pandemic  261.000000  261.000000  Default  27.0000  27.0000
2622  lockdown  313.000000  313.000000  Default  26.0000  26.0000
...        ...         ...         ...      ...      ...      ...
1773   friends   29.167404   59.793316  Topic10  -5.0183   1.9734
756      close   23.134265   34.087143  Topic10  -5.2500   2.3036
637      cause   22.663950   43.170546  Topic10  -5.2705   2.0468
3825       sad   22.117883   41.533810  Topic10  -5.2949   2.0611
1944     great   21.469910   74.393249  Topic10  -5.3247   1.4485

[509 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
3         3  0.972362       able
40        4  0.227782   actually
40        6  0.641931   actually
40        8  0.103537   actually
90        5  0.250624         ah
...     ...       ...        ...
4974      8  0.622741        yes
4975      1  0.976929  yesterday
4981     10  0.927532         yo
4998      8  0.119936       zoom
4998     10  0.875898       zoom

[756 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 7, 2, 8, 10, 6, 1, 9, 4, 3])

# Bigram Covid

In [9]:
vectorizer = TfidfVectorizer(
ngram_range=(2, 2),
analyzer='word',       
max_features=5000)# max number of unique words
                            
data_matrix = vectorizer.fit_transform(df_raw_covid['clean_tweet4'])

# Very slow!!!

lda_model = LatentDirichletAllocation(
n_components=10, # Number of topics
learning_method='online',
random_state=20,       
n_jobs = -1  # Use all available CPUs
                                     )
lda_output = lda_model.fit_transform(data_matrix)

import pyLDAvis.sklearn
pyLDAvis.enable_notebook()
pyLDAvis.sklearn.prepare(lda_model, data_matrix, vectorizer, mds='tsne')

/Users/jeromeyue/Desktop/omdena/lib/python3.9/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(
/Users/jeromeyue/Desktop/omdena/lib/python3.9/site-packages/sklearn/manifold/_t_sne.py:691: FutureWarning: 'square_distances' has been introduced in 0.24 to help phase out legacy squaring behavior. The 'legacy' setting will be removed in 1.1 (renaming of 0.26), and the default setting will be changed to True. In 1.3, 'square_distances' will be removed altogether, and distances will be squared by default. Set 'square_distances'=True to silence this warning.
  warnings.warn(


PreparedData(topic_coordinates=               x          y  topics  cluster       Freq
topic                                                  
6      32.420952   8.115230       1        1  10.677903
9       0.648955  71.707100       2        1  10.376353
0      71.019684   9.971006       3        1  10.079282
1       3.785981  33.185135       4        1   9.991141
5     -42.666500  -9.222070       5        1   9.966234
7      -3.621689 -43.526516       6        1   9.921002
4      -3.632402  -4.171708       7        1   9.888186
2     -36.584400  35.804413       8        1   9.807396
3      44.391380  50.615383       9        1   9.667443
8      40.235809 -31.591665      10        1   9.625059, topic_info=                   Term        Freq       Total Category  logprob  loglift
504     circuit breaker  323.000000  323.000000  Default  30.0000  30.0000
3910  social distancing  223.000000  223.000000  Default  29.0000  29.0000
4771          wear mask  134.000000  134.000000  Default  28.0000  28.0000
2901         new normal  102.000000  102.000000  Default  27.0000  27.0000
4779       wearing mask   99.000000   99.000000  Default  26.0000  26.0000
...                 ...         ...         ...      ...      ...      ...
2884        new cluster    9.027442    9.752902  Topic10  -5.5539   2.2635
4808          weeks ago    8.994549    9.720027  Topic10  -5.5576   2.2632
715         covid virus    8.927218    9.652708  Topic10  -5.5651   2.2627
4227         talk phone    8.922888    9.648340  Topic10  -5.5656   2.2626
2911           new york    8.689413    9.414877  Topic10  -5.5921   2.2606

[336 rows x 6 columns], token_table=      Topic      Freq           Term
term                                
67        1  0.951861     amid covid
80        6  0.977566    another day
139       5  0.972027   around world
209      10  0.950488      back home
213       8  0.924842    back office
...     ...       ...            ...
4985      4  0.971225     zoom calls
4986      1  0.976916     zoom class
4990      2  0.990245   zoom meeting
4991      9  0.963319  zoom meetings
4999      8  0.975971      zoom zoom

[311 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[7, 10, 1, 2, 6, 8, 5, 3, 4, 9])

# Trigram Covid

In [10]:
vectorizer = TfidfVectorizer(
ngram_range=(3, 3),
analyzer='word',       
max_features=5000)# max number of unique words
                            
data_matrix = vectorizer.fit_transform(df_raw_covid['clean_tweet4'])

# Very slow!!!

lda_model = LatentDirichletAllocation(
n_components=10, # Number of topics
learning_method='online',
random_state=20,       
n_jobs = -1  # Use all available CPUs
                                     )
lda_output = lda_model.fit_transform(data_matrix)

import pyLDAvis.sklearn
pyLDAvis.enable_notebook()
pyLDAvis.sklearn.prepare(lda_model, data_matrix, vectorizer, mds='tsne')

/Users/jeromeyue/Desktop/omdena/lib/python3.9/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(
/Users/jeromeyue/Desktop/omdena/lib/python3.9/site-packages/sklearn/manifold/_t_sne.py:691: FutureWarning: 'square_distances' has been introduced in 0.24 to help phase out legacy squaring behavior. The 'legacy' setting will be removed in 1.1 (renaming of 0.26), and the default setting will be changed to True. In 1.3, 'square_distances' will be removed altogether, and distances will be squared by default. Set 'square_distances'=True to silence this warning.
  warnings.warn(


PreparedData(topic_coordinates=               x          y  topics  cluster       Freq
topic                                                  
0      32.420952   8.115230       1        1  10.216769
5       0.648955  71.707100       2        1  10.171185
8      71.019684   9.971006       3        1  10.165278
9       3.785981  33.185135       4        1  10.165252
6     -42.666500  -9.222070       5        1  10.063409
7      -3.621689 -43.526516       6        1  10.060343
4      -3.632402  -4.171708       7        1   9.897759
3     -36.584400  35.804413       8        1   9.837534
2      44.391380  50.615383       9        1   9.763490
1      40.235809 -31.591665      10        1   9.658981, topic_info=                             Term       Freq      Total Category  logprob  \
3303            posted photo zoom  44.000000  44.000000  Default  30.0000   
3786    singapore circuit breaker  33.000000  33.000000  Default  29.0000   
658           circuit breaker day  33.000000  33.000000  Default  28.0000   
677        circuit breaker period  27.000000  27.000000  Default  27.0000   
969         day lockdown godbless  25.000000  25.000000  Default  26.0000   
...                           ...        ...        ...      ...      ...   
3581            reports new covid   3.267278   3.817715  Topic10  -5.5971   
3264  population fully vaccinated   3.250806   3.801243  Topic10  -5.6022   
3191              please help get   3.234185   3.784622  Topic10  -5.6073   
3304      posted video juggernaut   3.223991   3.774428  Topic10  -5.6104   
2589         marketing tools help   4.143892   8.387156  Topic10  -5.3594   

      loglift  
3303  30.0000  
3786  29.0000  
658   28.0000  
677   27.0000  
969   26.0000  
...       ...  
3581   2.1816  
3264   2.1809  
3191   2.1801  
3304   2.1797  
2589   1.6322  

[338 rows x 6 columns], token_table=      Topic      Freq                          Term
term                                               
2         9  0.766161             able help unsnarl
9         4  0.859784          activities week dose
18        4  0.859784  advises strenuous activities
24        6  0.904826                   ai iot aiot
33        2  0.961726        allowing family invest
...     ...       ...                           ...
4969      2  0.961726          wounds fro capricorn
4974      5  0.988211          yahoo singapore news
4979      9  0.933104                  year old boy
4986      3  0.969144         yesterday people died
4989      3  0.969144                 yin suu pusue

[311 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 6, 9, 10, 7, 8, 5, 4, 3, 2])

# Uni to Trigram Covid

In [11]:
vectorizer = TfidfVectorizer(
ngram_range=(1, 3),
analyzer='word',       
max_features=5000)# max number of unique words
                            
data_matrix = vectorizer.fit_transform(df_raw_covid['clean_tweet4'])

# Very slow!!!

lda_model = LatentDirichletAllocation(
n_components=10, # Number of topics
learning_method='online',
random_state=20,       
n_jobs = -1  # Use all available CPUs
                                     )
lda_output = lda_model.fit_transform(data_matrix)

import pyLDAvis.sklearn
pyLDAvis.enable_notebook()
pyLDAvis.sklearn.prepare(lda_model, data_matrix, vectorizer, mds='tsne')

/Users/jeromeyue/Desktop/omdena/lib/python3.9/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(
/Users/jeromeyue/Desktop/omdena/lib/python3.9/site-packages/sklearn/manifold/_t_sne.py:691: FutureWarning: 'square_distances' has been introduced in 0.24 to help phase out legacy squaring behavior. The 'legacy' setting will be removed in 1.1 (renaming of 0.26), and the default setting will be changed to True. In 1.3, 'square_distances' will be removed altogether, and distances will be squared by default. Set 'square_distances'=True to silence this warning.
  warnings.warn(


PreparedData(topic_coordinates=               x          y  topics  cluster       Freq
topic                                                  
4      32.420952   8.115230       1        1  13.488056
6       0.648955  71.707100       2        1  11.275608
0      71.019684   9.971006       3        1  10.290996
1       3.785981  33.185135       4        1  10.152956
9     -42.666500  -9.222070       5        1   9.970286
3      -3.621689 -43.526516       6        1   9.947116
7      -3.632402  -4.171708       7        1   9.544211
8     -36.584400  35.804413       8        1   9.221674
5      44.391380  50.615383       9        1   8.457352
2      40.235809 -31.591665      10        1   7.651745, topic_info=          Term        Freq       Total Category  logprob  loglift
4693   vaccine  319.000000  319.000000  Default  30.0000  30.0000
3102  pandemic  286.000000  286.000000  Default  29.0000  29.0000
4991      zoom  242.000000  242.000000  Default  28.0000  28.0000
4457     tired  515.000000  515.000000  Default  27.0000  27.0000
660    circuit  184.000000  184.000000  Default  26.0000  26.0000
...        ...         ...         ...      ...      ...      ...
3690      room   27.963354   34.210808  Topic10  -5.2206   2.3686
2500    lonely   41.178533   68.094482  Topic10  -4.8336   2.0673
4113      stay   51.765404  117.291646  Topic10  -4.6048   1.7523
2795   morning   34.804069   56.582935  Topic10  -5.0018   2.0843
407       body   25.626659   44.042424  Topic10  -5.3079   2.0287

[517 rows x 6 columns], token_table=      Topic      Freq          Term
term                               
32        6  0.344729      actually
32        8  0.646366      actually
38        8  0.974904     addiction
69        1  0.335751            ah
69        3  0.651752            ah
...     ...       ...           ...
4965      6  0.985981           yes
4966      9  0.984769     yesterday
4991      6  0.210341          zoom
4991     10  0.787748          zoom
4996      3  0.957300  zoom meeting

[803 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 7, 1, 2, 10, 4, 8, 9, 6, 3])